In [1]:
import requests
import psycopg2
from datetime import datetime

# Step 1: Fetch CPU usage from Prometheus
PROMETHEUS_URL = "http://192.168.0.99:9090/api/v1/query"
query = 'sum(rate(node_cpu_seconds_total{mode!="idle"}[5m]))'

response = requests.get(PROMETHEUS_URL, params={'query': query})
data = response.json()

if data['status'] == 'success' and data['data']['result']:
    cpu_value = float(data['data']['result'][0]['value'][1])
    print(f"Fetched CPU Usage: {cpu_value:.4f} cores")
else:
    raise ValueError("Failed to fetch data from Prometheus")

# Step 2: Store CPU usage into PostgreSQL
conn = psycopg2.connect(
    dbname='rag_db',
    user='user',
    password='password',
    host='192.168.0.99',
    port=5432
)

cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS daily_cpu_summary (
        date DATE PRIMARY KEY,
        total_cpu_used FLOAT
    );
""")
cur.execute("""
    INSERT INTO daily_cpu_summary (date, total_cpu_used)
    VALUES (%s, %s)
    ON CONFLICT (date) DO UPDATE
    SET total_cpu_used = EXCLUDED.total_cpu_used;
""", (datetime.utcnow().date(), cpu_value))

conn.commit()
conn.close()
print("CPU data fetched from Prometheus and inserted into DB successfully.")


Fetched CPU Usage: 0.3960 cores
CPU data fetched from Prometheus and inserted into DB successfully.
